In [1]:
# setup
!pip install geopandas rasterio earthengine-api numpy pandas matplotlib scikit-learn scipy

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import ee
from datetime import datetime
import urllib.request
import zipfile
from scipy import stats
from shapely.geometry import Point, box

# authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='colonia-detection')

# load the sample data from previous notebook
zip_url = "https://github.com/wvg1/colonia-detection/archive/refs/heads/main.zip"
urllib.request.urlretrieve(zip_url, 'repo.zip')

with zipfile.ZipFile('repo.zip', 'r') as zip_ref:
    zip_ref.extractall()

shapefile_path = 'colonia-detection-main/data/raw/colonias_shapefile/NewMexicoCandidateColoniaBlocks.shp'
colonias_gdf = gpd.read_file(shapefile_path)
colonias_gdf = colonias_gdf.to_crs('EPSG:4326')

# group by colonia
colonias_grouped = colonias_gdf.groupby('Colonia').apply(
    lambda x: gpd.GeoSeries(x.geometry).union_all()
).reset_index()

colonias_grouped.columns = ['Colonia', 'geometry']
colonias_grouped = gpd.GeoDataFrame(colonias_grouped, crs='EPSG:4326')
colonias_grouped['type'] = 'colonia'

print(f"Colonias loaded: {len(colonias_grouped)}")

Colonias loaded: 80


/tmp/ipython-input-953319281.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  colonias_grouped = colonias_gdf.groupby('Colonia').apply(


In [2]:
# calculate spectral indices for colonias
def extract_indices(geometry):
    """extract mean spectral indices for a geometry"""

    sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(geometry) \
        .filterDate('2024-01-01', '2024-12-31') \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .sort('CLOUD_COVERAGE_ASSESSMENT') \
        .first()

    nir = sentinel2.select('B8')
    red = sentinel2.select('B4')
    swir = sentinel2.select('B11')

    ndvi = nir.subtract(red).divide(nir.add(red))
    ndbi = swir.subtract(nir).divide(swir.add(nir))
    ndmi = nir.subtract(swir).divide(nir.add(swir))

    features = {
        'ndvi': ndvi.reduceRegion(ee.Reducer.mean(), geometry, 30).getInfo(),
        'ndbi': ndbi.reduceRegion(ee.Reducer.mean(), geometry, 30).getInfo(),
        'ndmi': ndmi.reduceRegion(ee.Reducer.mean(), geometry, 30).getInfo(),
    }

    return features

print("extracting features for colonias...")
colonia_features = []

for idx, row in colonias_grouped.iterrows():
    try:
        # convert shapely geometry to ee.Geometry, handles both Polygon and MultiPolygon
        if row.geometry.geom_type == 'Polygon':
            coords = list(row.geometry.exterior.coords)
            geom_ee = ee.Geometry.Polygon(coords)
        elif row.geometry.geom_type == 'MultiPolygon':
            # for MultiPolygon, use the first polygon or create a MultiPolygon
            coords = list(row.geometry.geoms[0].exterior.coords)
            geom_ee = ee.Geometry.Polygon(coords)
        else:
            # fallback: use the geo_interface but wrap coordinates properly
            geo_dict = row.geometry.__geo_interface__
            geom_ee = ee.Geometry(geo_dict)

        features = extract_indices(geom_ee)

        colonia_features.append({
            'Colonia': row['Colonia'],
            'type': 'colonia',
            'ndvi': list(features['ndvi'].values())[0] if features['ndvi'] else np.nan,
            'ndbi': list(features['ndbi'].values())[0] if features['ndbi'] else np.nan,
            'ndmi': list(features['ndmi'].values())[0] if features['ndmi'] else np.nan,
        })
        print(f"  {row['Colonia']}: done")
    except Exception as e:
        print(f"  {row['Colonia']}: error - {e}")

colonia_df = pd.DataFrame(colonia_features)
print(f"\nextracted features for {len(colonia_df)} colonias")
print(colonia_df.head())

extracting features for colonias...
  Alma: done
  Anthony: done
  Arenas Valley: done
  Bear Mountain: done
  Beaverhead: done
  Bent: done
  Berino: done
  Brazito: done
  Buckhorn: done
  Carlisle: done
  Catfish Farms: done
  Cattleland: done
  Chamberino: done
  Chaparral: done
  City of Sunland Park: done
  Cottage San: done
  Cotton: done
  Del Cerro: done
  Del Sol: done
  Dog Canyon: done
  Dona Ana: done
  Dungan: done
  El Milagro: done
  Fairacres: done
  Faywood: done
  Ft. Selden: done
  Garfield: done
  Gila Hot Springs: done
  Glen Acres: done
  Glenwood: done
  Hachita: done
  High Rolls: done
  Hill: done
  Joy Drive Subd.: done
  Keeler Farm Road: done
  La Mesa: done
  La Union: done
  Lake Roberts: done
  Las Palmeras: done
  Leasburg: done
  Luna: done
  Mayhill: done
  Mesquite: done
  Mimbres: done
  Mockingbird Hill: done
  Mogollon: done
  Montana Vista: done
  Moongate: done
  Mountain View: done
  Mule Creek: done
  Nogal: done
  Old Picacho: done
  Organ: d

In [3]:
# reload sample dataset
bounds = colonias_grouped.total_bounds

np.random.seed(42)
num_controls = len(colonias_grouped) * 2

random_lons = np.random.uniform(bounds[0], bounds[2], num_controls)
random_lats = np.random.uniform(bounds[1], bounds[3], num_controls)

control_points = gpd.GeoDataFrame(
    {'type': ['control'] * num_controls},
    geometry=[Point(lon, lat) for lon, lat in zip(random_lons, random_lats)],
    crs='EPSG:4326'
)

# remove controls inside colonias
control_points = gpd.sjoin(
    control_points,
    colonias_grouped[['geometry']],
    how='left',
    predicate='within'
)
control_points = control_points[control_points.index_right.isna()]
control_points = control_points[['type', 'geometry']].head(num_controls)

# combine colonias and controls
colonias_grouped['type'] = 'colonia'
sample_data = pd.concat([
    colonias_grouped[['type', 'geometry']],
    control_points
], ignore_index=True)

print(f"Sample dataset:")
print(f"Colonias: {(sample_data['type'] == 'colonia').sum()}")
print(f"Controls: {(sample_data['type'] == 'control').sum()}")

Sample dataset:
Colonias: 80
Controls: 152


In [4]:
# calculate all spectral indices with error handling
def extract_indices(geometry):
    """extract mean spectral indices for a geometry"""

    sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(geometry) \
        .filterDate('2024-01-01', '2024-12-31') \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .sort('CLOUD_COVERAGE_ASSESSMENT') \
        .first()

    # select all bands needed
    blue = sentinel2.select('B2')
    green = sentinel2.select('B3')
    red = sentinel2.select('B4')
    nir = sentinel2.select('B8')
    swir = sentinel2.select('B11')

    # core indices
    ndvi = nir.subtract(red).divide(nir.add(red))
    ndbi = swir.subtract(nir).divide(swir.add(nir))
    ndmi = nir.subtract(swir).divide(nir.add(swir))

    # additional indices (calculate each separately to isolate errors)
    try:
        bsi = swir.add(red).subtract(nir.add(blue)).divide(swir.add(red).add(nir.add(blue)))
    except:
        bsi = None

    try:
        ndwi = green.subtract(nir).divide(green.add(nir))
    except:
        ndwi = None

    try:
        ibi = ndbi.multiply(2).divide(ndbi.add(ndvi).add(ndmi))
    except:
        ibi = None

    features = {
        'ndvi': ndvi.reduceRegion(ee.Reducer.mean(), geometry, 30).getInfo(),
        'ndbi': ndbi.reduceRegion(ee.Reducer.mean(), geometry, 30).getInfo(),
        'ndmi': ndmi.reduceRegion(ee.Reducer.mean(), geometry, 30).getInfo(),
        'bsi': bsi.reduceRegion(ee.Reducer.mean(), geometry, 30).getInfo() if bsi else {},
        'ndwi': ndwi.reduceRegion(ee.Reducer.mean(), geometry, 30).getInfo() if ndwi else {},
        'ibi': ibi.reduceRegion(ee.Reducer.mean(), geometry, 30).getInfo() if ibi else {},
    }

    return features

# extract spectral indices for both colonias and control points (all 6 indices)
print("extracting features for colonias and controls...")
all_features = []

for idx, row in sample_data.iterrows():
    try:
        # convert shapely geometry to ee.Geometry
        if row.geometry.geom_type == 'Polygon':
            coords = list(row.geometry.exterior.coords)
            geom_ee = ee.Geometry.Polygon(coords)
        elif row.geometry.geom_type == 'MultiPolygon':
            coords = list(row.geometry.geoms[0].exterior.coords)
            geom_ee = ee.Geometry.Polygon(coords)
        elif row.geometry.geom_type == 'Point':
            # for point geometries, create a small buffer
            geom_ee = ee.Geometry.Point([row.geometry.x, row.geometry.y]).buffer(100)
        else:
            geo_dict = row.geometry.__geo_interface__
            geom_ee = ee.Geometry(geo_dict)

        features = extract_indices(geom_ee)

        # safely extract values from dictionaries
        def safe_extract(feat_dict):
            try:
                if feat_dict:
                    vals = list(feat_dict.values())
                    return vals[0] if vals else np.nan
                return np.nan
            except:
                return np.nan

        all_features.append({
            'id': idx,
            'type': row['type'],
            'ndvi': safe_extract(features['ndvi']),
            'ndbi': safe_extract(features['ndbi']),
            'ndmi': safe_extract(features['ndmi']),
            'bsi': safe_extract(features['bsi']),
            'ndwi': safe_extract(features['ndwi']),
            'ibi': safe_extract(features['ibi']),
        })

        if (idx + 1) % 20 == 0:
            print(f"  processed {idx + 1} / {len(sample_data)} samples")

    except Exception as e:
        print(f"  id {idx} ({row['type']}): error - {e}")

sample_df = pd.DataFrame(all_features)
print(f"\nextracted features for {len(sample_df)} samples")
print(f"Colonias: {(sample_df['type'] == 'colonia').sum()}")
print(f"Controls: {(sample_df['type'] == 'control').sum()}")

# check for missing data
print("Missing data check:")
print(sample_df[['ndvi', 'ndbi', 'ndmi', 'bsi', 'ndwi', 'ibi']].isnull().sum())
print("\nPercentage missing by index:")
print((sample_df[['ndvi', 'ndbi', 'ndmi', 'bsi', 'ndwi', 'ibi']].isnull().sum() / len(sample_df) * 100).round(2))
print("\nRows with any missing values:")
print(sample_df[['ndvi', 'ndbi', 'ndmi', 'bsi', 'ndwi', 'ibi']].isnull().any(axis=1).sum())

print("Summary statistics by type:")
print(sample_df.groupby('type')[['ndvi', 'ndbi', 'ndmi', 'bsi', 'ndwi', 'ibi']].describe())

extracting features for colonias and controls...


  processed 20 / 232 samples
  processed 40 / 232 samples
  processed 60 / 232 samples
  processed 80 / 232 samples
  processed 100 / 232 samples
  processed 120 / 232 samples
  processed 140 / 232 samples
  processed 160 / 232 samples
  processed 180 / 232 samples
  processed 200 / 232 samples
  processed 220 / 232 samples

extracted features for 232 samples
Colonias: 80
Controls: 152

Summary statistics by type:
          ndvi                                                             \
         count     mean       std       min       25%       50%       75%   
type                                                                        
colonia   80.0  0.23754  0.114562  0.026659  0.129757  0.224020  0.309354   
control  152.0  0.19190  0.112631  0.079401  0.122493  0.146609  0.218853   

                    ndbi            ...      ndwi              ibi  \
              max  count      mean  ...       75%       max  count   
type                                ...                 

In [ ]:
# look for ideal date
def find_best_date(geometry, start_date, end_date, cloud_threshold=20):
    """Find the best (least cloudy) image over a date range for a geometry"""
    sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(geometry) \
        .filterDate(start_date, end_date) \
        .sort('CLOUDY_PIXEL_PERCENTAGE')

    best_image = sentinel2.first()
    cloud_cover = ee.Number(best_image.get('CLOUDY_PIXEL_PERCENTAGE')).getInfo()
    image_date = ee.Date(best_image.get('system:time_start')).format('YYYY-MM-DD').getInfo()

    return image_date, cloud_cover

# check best possible date for each sample
print("Finding best dates for all samples...")
date_cloud_check = []

for idx, row in sample_data.iterrows():
    # convert geometry
    if row.geometry.geom_type == 'Polygon':
        coords = list(row.geometry.exterior.coords)
        geom_ee = ee.Geometry.Polygon(coords)
    elif row.geometry.geom_type == 'MultiPolygon':
        coords = list(row.geometry.geoms[0].exterior.coords)
        geom_ee = ee.Geometry.Polygon(coords)
    else:
        geom_ee = ee.Geometry.Point([row.geometry.x, row.geometry.y]).buffer(100)

    try:
        best_date, cloud_cover = find_best_date(geom_ee, '2024-01-01', '2024-12-31')
        date_cloud_check.append({
            'id': idx,
            'type': row['type'],
            'best_date': best_date,
            'cloud_cover': cloud_cover
        })
    except Exception as e:
        print(f"Error checking {idx}: {e}")

date_check_df = pd.DataFrame(date_cloud_check)

print("\n" + "="*80)
print("Cloud cover summary:")
print("="*80)
print(date_check_df.groupby('type')['cloud_cover'].describe())
print(f"\nSamples with cloud cover < 20%:")
print(date_check_df[date_check_df['cloud_cover'] < 20].groupby('type').size())
print(f"\nTotal samples: {len(date_check_df)}")

Finding best dates for all samples...


In [ ]:
from scipy import stats
import pandas as pd

# prepare data for testing
indices = ['ndvi', 'ndbi', 'ndmi', 'bsi', 'ndwi', 'ibi']

colonias = sample_df[sample_df['type'] == 'colonia']
controls = sample_df[sample_df['type'] == 'control']

# run statistical tests
print("="*80)
print("STATISTICAL TESTS: COLONIAS vs CONTROLS")
print("="*80)

results = []

for idx in indices:
    colonia_vals = colonias[idx].dropna()
    control_vals = controls[idx].dropna()

    # t-test (parametric)
    t_stat, t_pval = stats.ttest_ind(colonia_vals, control_vals)

    # Mann-Whitney U test (non-parametric)
    u_stat, u_pval = stats.mannwhitneyu(colonia_vals, control_vals)

    # Effect size (Cohen's d)
    pooled_std = np.sqrt(((len(colonia_vals)-1)*colonia_vals.std()**2 +
                           (len(control_vals)-1)*control_vals.std()**2) /
                          (len(colonia_vals) + len(control_vals) - 2))
    cohens_d = (colonia_vals.mean() - control_vals.mean()) / pooled_std

    results.append({
        'Index': idx.upper(),
        'Colonia Mean': colonia_vals.mean(),
        'Control Mean': control_vals.mean(),
        'Difference': colonia_vals.mean() - control_vals.mean(),
        't-statistic': t_stat,
        't-pvalue': t_pval,
        'U-statistic': u_stat,
        'U-pvalue': u_pval,
        "Cohen's d": cohens_d,
        'Significant (p<0.05)': 'YES' if t_pval < 0.05 else 'NO'
    })

results_df = pd.DataFrame(results)

# display results
print("\nDetailed Results:")
print(results_df.to_string(index=False))

# summary table
print("\n" + "="*80)
print("SUMMARY: Which indices best distinguish colonias from controls?")
print("="*80)

summary = results_df[['Index', 'Colonia Mean', 'Control Mean', 'Difference',
                       "Cohen's d", 'Significant (p<0.05)']].copy()
summary = summary.sort_values("Cohen's d", key=abs, ascending=False)

print(summary.to_string(index=False))

# interpretation
print("\n" + "="*80)
print("INTERPRETATION:")
print("="*80)

sig_indices = results_df[results_df['Significant (p<0.05)'] == 'YES']
print(f"\nStatistically significant differences (p < 0.05): {len(sig_indices)} / {len(indices)} indices")

for _, row in sig_indices.iterrows():
    direction = "HIGHER" if row['Difference'] > 0 else "LOWER"
    print(f"\n{row['Index']}:")
    print(f"  - Colonias have {direction} values than controls")
    print(f"  - Colonias: {row['Colonia Mean']:.4f} | Controls: {row['Control Mean']:.4f}")
    print(f"  - Effect size (Cohen's d): {row['Cohen\'s d']:.4f}")
    if abs(row['Cohen\'s d']) > 0.8:
        effect = "LARGE"
    elif abs(row['Cohen\'s d']) > 0.5:
        effect = "MEDIUM"
    else:
        effect = "SMALL"
    print(f"  - Effect: {effect}")


STATISTICAL TESTS: COLONIAS vs CONTROLS

Detailed Results:
Index  Colonia Mean  Control Mean  Difference  t-statistic  t-pvalue  U-statistic     U-pvalue  Cohen's d Significant (p<0.05)
 NDVI      0.237540      0.191900    0.045640     2.916380  0.003892       7752.0 5.818195e-04   0.402830                  YES
 NDBI      0.032017      0.107635   -0.075618    -4.729369  0.000004       3251.0 5.847565e-09  -0.653251                  YES
 NDMI     -0.032017     -0.107635    0.075618     4.729369  0.000004       8909.0 5.847565e-09   0.653251                  YES
  BSI      0.102534      0.160757   -0.058222    -4.850865  0.000002       3386.0 2.969434e-08  -0.670033                  YES
 NDWI     -0.344898     -0.317536   -0.027361    -2.269824  0.024145       4638.0 3.011022e-03  -0.313523                  YES
  IBI    327.369878      1.761140  325.608737     1.378543  0.169375       4325.0 3.052950e-04   0.190413                   NO

SUMMARY: Which indices best distinguish colonias fr